# Supervised learning- regression- knn and penalisation

# 0. Loading packages and dataset

In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')

df = pd.read_csv('insurance_cleaned.csv')

C:\Users\tomas\AppData\Local\Temp\ipykernel_23688\2495569210.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


## 0.1. Train/test-split

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from random import Random

df_shuffle = df.sample(frac=1, random_state=40)

X = df_shuffle.drop(['charges'],1)
y = df_shuffle['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

num = ['age','bmi','children']
scaler = StandardScaler()
X_train_stand = X_train.copy()
X_test_stand = X_test.copy()
X_train_stand[num] = scaler.fit_transform(X_train[num])
X_test_stand[num] = scaler.transform(X_test[num])

C:\Users\tomas\AppData\Local\Temp\ipykernel_23688\1196696295.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df_shuffle.drop(['charges'],1)


In [10]:
X_train_stand.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
217,-0.870587,-1.225955,-0.901777,0,1,1,0,0,0,1,0
1060,-1.084879,0.234041,-0.901777,0,1,1,0,0,0,1,0
555,-0.799156,-1.111253,0.740608,0,1,1,0,0,0,0,1
1155,-0.227711,-1.384080,1.561801,1,0,1,0,1,0,0,0
888,-1.227741,1.461355,-0.901777,0,1,1,0,0,0,0,1


In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1070 entries, 217 to 52
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1070 non-null   int64  
 1   bmi               1070 non-null   float64
 2   children          1070 non-null   int64  
 3   sex_female        1070 non-null   int64  
 4   sex_male          1070 non-null   int64  
 5   smoker_no         1070 non-null   int64  
 6   smoker_yes        1070 non-null   int64  
 7   region_northeast  1070 non-null   int64  
 8   region_northwest  1070 non-null   int64  
 9   region_southeast  1070 non-null   int64  
 10  region_southwest  1070 non-null   int64  
dtypes: float64(1), int64(10)
memory usage: 100.3 KB


# 1. Decision trees

We start with a simple decision tree, but use cross-validation with regard to the maximum depth to find the best model. Since there are 1070 observation, the depth can't be higher than 11.

In [ ]:
from sklearn.tree import DecisionTreeRegressor  
from sklearn.model_selection import cross_val_score

depth = np.arange(1,11)    # This will give an array of numbers between 1 and 10
cv_scores = []
sd_scores = []
# perform 5-fold cross validation on the  possible values for the radius (bandwith)
for d in depth:
    dec_tree = DecisionTreeRegressor(random_state = 0, max_depth=d)  
    scores = cross_val_score(dec_tree, X_train_stand, y_train,  cv=5)
    cv_scores.append(scores.mean())
    sd_scores.append(np.sqrt(scores.var())/np.sqrt(5))
 
max_value = max(cv_scores)
max_index = cv_scores.index(max_value)

plt.plot(depth, cv_scores)
plt.xlabel('depth')
plt.ylabel('R^2')

plt.show()
print('The best depth is', depth[max_index])
print('The validated score is', max_value)

5 is the best model. The model that is more simple (a depth of 3), is above the underlimit, so we choose 3.

In [ ]:
from sklearn import tree  
dec_tree = DecisionTreeRegressor(random_state = 0, max_depth=depth[max_index])  
dec_tree.fit(X_train_stand, y_train) 
print(dec_tree.score(X_train_stand, y_train) )
print(max_value)

# 2. Random forest
Let's see if we can improve this by using random forests. 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

criterion =['squared_error','absolute_error']
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 400)]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [int(x) for x in np.linspace(2, 10, num = 9)]
min_samples_leaf = [int(x) for x in np.linspace(1, 10, num = 10)]
max_leaf_nodes = [int(x) for x in np.linspace(10, 500, num = 490)]

# create the random grid to search for best hyperparameters
random_grid = {'criterion': criterion,
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_leaf_nodes': max_leaf_nodes}

# then do cross-validatoin
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=5, random_state=42, n_jobs=-1)
# n_jobs=-1 to run as many models  parallel as possible
rf_random.fit(X_train_stand, y_train)



In [ ]:
print(rf_random.score(X_train_stand, y_train)) # training score
print(rf_random.best_score_)  # validation score

In [ ]:
rf_random.best_params_

In [ ]:
out = pd.DataFrame(rf_random.cv_results_)
out.head()

In [ ]:

xlabel_names = ['param_max_depth','param_min_samples_split','param_min_samples_leaf','param_n_estimators',
                'param_max_features', 'param_max_leaf_nodes', 'param_criterion']

fig, axs = plt.subplots(3,3, figsize=(20,10))

axs[0,0].scatter(out['param_max_depth'], out['mean_test_score'], c='blue');
axs[0,0].set_title('max_depth')

axs[0,1].scatter(out['param_min_samples_split'], out['mean_test_score'], c='blue');
axs[0,1].set_title('min_samples_split')

axs[0,2].scatter(out['param_min_samples_leaf'], out['mean_test_score'], c='blue');
axs[0,2].set_title('min_samples_leaf')

axs[1,0].scatter(out['param_n_estimators'], out['mean_test_score'], c='blue');
axs[1,0].set_title('n_estimators')

axs[1,1].scatter(out['param_max_features'], out['mean_test_score'], c='blue');
axs[1,1].set_title('max_features')

axs[1,2].scatter(out['param_max_leaf_nodes'], out['mean_test_score'], c='blue');
axs[1,2].set_title('max_leaf_nodes')

axs[2,0].scatter(out['param_criterion'], out['mean_test_score'], c='blue');
axs[2,0].set_title('criterion')

for ax in axs.flat:
    ax.set(ylabel='r_squared')
    

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(out['param_max_depth'], out['mean_test_score'], c='blue');
ax.set_xlabel("max_depth");
ax.set_ylabel("r_squared");

We can see that the best depth is 7, but 6, 9 and 10 do not perform much worse. However, we prefer models that are less complex. Here, that means models that are less deep. So, we chose 7 and 6 to go in the grid search.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(out['param_min_samples_split'], out['mean_test_score'], c='blue');
ax.set_xlabel("min_samples_split");
ax.set_ylabel("r_squared");

We can see that the best value is 10, but 2,5,6, 8,9 and 10 do not perform much worse. However, we prefer models that are less complex. Here, that means models that have a higher min number of samples to allow splitting. So, we chose 10,11 and 12 to go in the grid search.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(out['param_min_samples_leaf'], out['mean_test_score'], c='blue');
ax.set_xlabel("min_samples_leaf");
ax.set_ylabel("r_squared");

5 leads to the best R^2, but 8 and 10 are not that much worse. We prefer a higher number of samples in a leaf (less complex), thus we include 5, 8 and 10.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(out['param_n_estimators'], out['mean_test_score'], c='blue');
ax.set_xlabel("n_estimators");
ax.set_ylabel("r_squared");

The best value was with 236 trees. Let's add  234,235, 236,237 and 238 to the grid search.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(out['param_max_features'], out['mean_test_score'], c='blue');
ax.set_xlabel("max_features");
ax.set_ylabel("r_squared");

Auto clearly wins here, so let's just go with that.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(out['param_max_leaf_nodes'], out['mean_test_score'], c='blue');
ax.set_xlabel("max_leaf_nodes");
ax.set_ylabel("r_squared");

The best max_leaf_nodes was 410, but it all seems very similar. Let's just not restrict this in the grid search.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(out['param_criterion'], out['mean_test_score'], c='blue');
ax.set_xlabel("criterion");
ax.set_ylabel("r_squared");

mae (absolute error) had the best score. we will include only mae.

Note: add more values to your grid for the project. I really limited myself here, so the model would not run too long for the demo

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

criterion =['absolute_error']
n_estimators = [234,235, 236,237,238]
max_features = ['auto']
max_depth = [6,7]
min_samples_split = [10,11,12]
min_samples_leaf = [5,8,10]

# create the random grid to search for best hyperparameters
grid = {'criterion': criterion,
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

# then do cross-validatoin
rf = RandomForestRegressor()
rf_grid = GridSearchCV(estimator = rf, param_grid = grid,
                               cv = 5, verbose=2,  n_jobs=-1)
# n_jobs=-1 to run as many models  parallel as possible
rf_grid.fit(X_train_stand, y_train)

In [ ]:
print(rf_grid.score(X_train_stand, y_train)) 
print(rf_grid.best_score_)

In [ ]:
rf_grid.best_params_

Not a lot has changed, the validation score improved a bit.
We retrain the model with the best parameters on the whole training set, before we take a look at whitening the black box

In [ ]:
params = rf_grid.best_params_
rf_gridBest = RandomForestRegressor(**params)
rf_gridBest.fit(X_train_stand, y_train)


# 3. Whitening the black box

Random forest is good all-round algorithm. It may not always be the very best, but it will almost always lead to a very reasonable performance, whithout a lot of effort. However, in linear regression we had the beta-parameters, which are easy to interpret. In random forest, it becomes a lot harder to interpret your results. This is why machine learning is often referred to as a black box. However, there are ways to whiten this black box. We will try to find an anwer for three questions
1. Which features are important?
2. What is the influence of a certain feature?
3. Why is a prediction what it is?	

## 3.1 Which features are important?

You can ask for the variable importance by using the function .feature_importances_

In [ ]:
list(zip(X_train.columns, rf_gridBest.feature_importances_))

This is not easy to read, so it is better to plot this. But first, let's check if the sum of all feature importances is indeed 1.

In [ ]:
rf_gridBest.feature_importances_.sum()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20,10))

feat_importances = pd.Series(rf_gridBest.feature_importances_, index=X.columns)
feat_importances.nlargest(11).plot(kind='barh')
plt.show()


So, the five features that seem to have the most influence in our algorithm are the number of children, bmi, not smoking, smoking and age.

But in what is the direction of this influence? Here, you might guess it from the context ( being older will probably result in higher medical costs), but be carefull with making assumptions like that. There is always a chance that you are wrong. So you have to check the directions of the influences yourself.

Another way to check the importance of features is to calculate the permutation importance using the eli5-package. This permutes the values of each feature one by one and checks how it changes model performance.

## 3.2 What is the influence of a certain feature?

Now that we know which are the important features, we can try to figure our how they are related to the outcome.  To this end, we build a regression on a set of predictions. It really doesn't matter in which set you calculate this, as long as it is representative of the population you want to discuss. Here, we do it just for the train set.

The first step is to get the predictions from the random forest.

In [ ]:

pred_train =rf_gridBest.predict(X_train_stand)

We then perform one univariate linear regressions per feature on the predicted values!

We use the standardized features and also standardize the predictions, to make the interpretation easier (will be a number between -1 and 1). 

In [ ]:
from sklearn.linear_model import LinearRegression

scaler2 = StandardScaler()
X_train_stand2 = X_train.copy()
X_train_stand2 = scaler.fit_transform(X_train)

X = pd.DataFrame(X_train_stand2)
X.columns = X_train.columns
predictors =  X_train.columns


y = np.array((pred_train-pred_train.mean())/np.sqrt(pred_train.var())).reshape(-1, 1)

for i in np.arange(1,(X.shape[1])):
    reg = LinearRegression().fit(X[[predictors[i]]], y)
    beta_help = pd.Series(reg.coef_[0])
    names_help = pd.Series(predictors[i])
    beta = pd.concat([beta,beta_help], axis=0)
    names = pd.concat([names,names_help], axis=0)
betas = pd.concat([names,beta],axis=1)
betas



We can see that how older you are, the higher the costs; having more children lead to higher costs; being female leads to lower costs, while being male then ofcourse leads to higher costs.

## 3.3 Why is a prediction what it is?

Now say we have a prediction that seems odd (e.g the prediction is far of the true value). You would like to know how your model came to that prediction.



In [ ]:
import lime
import lime.lime_tabular
from lime.lime_tabular import LimeTabularExplainer


# creating the explainer function
explainer = LimeTabularExplainer(X_train_stand.values, mode="regression", feature_names=X_train.columns)

# storing a new observation
i = 100
X_test_stand = pd.DataFrame(X_test_stand)
X_test_stand.columns = X_train.columns

X_observation = X_test_stand.iloc[[i], :]

In [ ]:
rf_gridBest.predict(X_observation)

In [ ]:
# explanation using the random forest model
explanation = explainer.explain_instance(X_observation.values[0], rf_gridBest.predict)
explanation.show_in_notebook(show_table=True, show_all=False)


We can see that the prediction for this point is 9.78 (middle of the left bar). By permuting the values of the different features, the prediction ranged from 7.29 to 10.79. In the table on the right, we can see the point itself. In the graph in the middle, you can see how the features influence the point. So, if you are a smoker (smoker_yes > 0), this will have a positive impact on the prediction (the prediction will be higher with 0.80). If the standardized age is lower than -0.87, the prediction will be lower with 0.80

# 4. Gradient boosting
Let's add one last algorithm for regression. After this, we will move on to classification.


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor


n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 50)]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [int(x) for x in np.linspace(2, 10, num = 9)]
min_samples_leaf = [int(x) for x in np.linspace(1, 10, num = 10)]
max_leaf_nodes = [int(x) for x in np.linspace(10, 500, num = 50)]
loss = ['ls','lad','huber','quantile']
learning_rate = [round(x,5) for x in np.logspace(start = -3, stop = -0.01, num = 20)]

# create the random grid to search for best hyperparameters
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_leaf_nodes': max_leaf_nodes,
               'loss': loss,
               'learning_rate': learning_rate}

# then do cross-validatoin
gbm = GradientBoostingRegressor()
gbm_random = RandomizedSearchCV(estimator = gbm, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=2, random_state=42, n_jobs=-1)
gbm_random.fit(X_train_stand, y_train)


In [ ]:
print(gbm_random.score(X_train_stand, y_train))
print(gbm_random.best_score_)

In [ ]:
gbm_random.best_params_

In [ ]:
out = pd.DataFrame(gbm_random.cv_results_)
out.head()

In [ ]:
out.mean_test_score 

In [ ]:
out2 = out


xlabel_names = ['param_max_depth','param_min_samples_split','param_min_samples_leaf','param_n_estimators',
                'param_max_features', 'param_max_leaf_nodes']

fig, axs = plt.subplots(3,3, figsize=(20,10))

axs[0,0].scatter(out2['param_max_depth'], out2['mean_test_score'], c='blue');
axs[0,0].set_title('max_depth')

axs[0,1].scatter(out2['param_min_samples_split'], out2['mean_test_score'], c='blue');
axs[0,1].set_title('min_samples_split')

axs[0,2].scatter(out2['param_min_samples_leaf'], out2['mean_test_score'], c='blue');
axs[0,2].set_title('min_samples_leaf')

axs[1,0].scatter(out2['param_n_estimators'], out2['mean_test_score'], c='blue');
axs[1,0].set_title('n_estimators')

axs[1,1].scatter(out2['param_max_features'], out2['mean_test_score'], c='blue');
axs[1,1].set_title('max_features')

axs[1,2].scatter(out2['param_max_leaf_nodes'], out2['mean_test_score'], c='blue');
axs[1,2].set_title('max_leaf_nodes')

axs[2,1].scatter(out2['param_learning_rate'], out2['mean_test_score'], c='blue');
axs[2,1].set_title('learning_rate')

axs[2,2].scatter(out2['param_loss'], out2['mean_test_score'], c='blue');
axs[2,2].set_title('loss')


for ax in axs.flat: ax.set(ylabel='r_squared')

It is hard to see anything on these graphs, because of some very low scores. We will zoom in on the R^2 above 0.7

In [ ]:
out2 = out[out.mean_test_score > 0.7]


xlabel_names = ['param_max_depth','param_min_samples_split','param_min_samples_leaf','param_n_estimators',
                'param_max_features', 'param_max_leaf_nodes', 'param_criterion']

fig, axs = plt.subplots(3,3, figsize=(20,10))

axs[0,0].scatter(out2['param_max_depth'], out2['mean_test_score'], c='blue');
axs[0,0].set_title('max_depth')

axs[0,1].scatter(out2['param_min_samples_split'], out2['mean_test_score'], c='blue');
axs[0,1].set_title('min_samples_split')

axs[0,2].scatter(out2['param_min_samples_leaf'], out2['mean_test_score'], c='blue');
axs[0,2].set_title('min_samples_leaf')

axs[1,0].scatter(out2['param_n_estimators'], out2['mean_test_score'], c='blue');
axs[1,0].set_title('n_estimators')

axs[1,1].scatter(out2['param_max_features'], out2['mean_test_score'], c='blue');
axs[1,1].set_title('max_features')

axs[1,2].scatter(out2['param_max_leaf_nodes'], out2['mean_test_score'], c='blue');
axs[1,2].set_title('max_leaf_nodes')


axs[2,1].scatter(out2['param_learning_rate'], out2['mean_test_score'], c='blue');
axs[2,1].set_title('learning_rate')

axs[2,2].scatter(out2['param_loss'], out2['mean_test_score'], c='blue');
axs[2,2].set_title('loss')


for ax in axs.flat: ax.set(ylabel='r_squared')
    

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

n_estimators = [172,173,174]
max_features = ['auto']
max_depth = [4,5]
min_samples_split = [5,7]
min_samples_leaf = [7,10]
max_leaf_nodes = [129,130,131]
learning_rate = [(x) for x in np.linspace(0.01, 0.05, num = 3)]
loss = ['huber']

# create the random grid to search for best hyperparameters
grid = {
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
            'max_leaf_nodes': max_leaf_nodes,
               'learning_rate': learning_rate,
               'loss':loss}

# then do cross-validatoin
gbm = GradientBoostingRegressor()
gbm_grid = GridSearchCV(estimator = gbm, param_grid = grid,
                               cv = 3, verbose=2,  n_jobs=-1)
# n_jobs=-1 to run as many models  parallel as possible
gbm_grid.fit(X_train_stand, y_train)

In [ ]:
gbm_grid.best_params_

In [ ]:
gbm_grid.best_score_

This is the best validated score so far. We choose this model. We retrain the model on the whole training set and evaluate on the test set.

In [ ]:
params = gbm_grid.best_params_
gbm_gridBest = GradientBoostingRegressor(**params)
gbm_gridBest.fit(X_train_stand, y_train)
print('R2: %.3f' % gbm_gridBest.score(X_train_stand, y_train)) # train score
print('R2: %.3f' % gbm_gridBest.score(X_test_stand, y_test))  # test score

 We will end it here for supervised regression. There are many, many more algorithms, but you got the main idea of how to do machine learning for regression. You can now easily learn other algorithms if you want to. 

# 5. Adaboost & XGBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score
adareg = AdaBoostRegressor(random_state = 123, n_estimators=10)
adareg.fit(X_train_stand, y_train)
pred = adareg.predict(X_test_stand)
adareg_r2 = r2_score(y_test, pred)
print("R2:", adareg_r2)

In [ ]:
residuals = y_test - pred

In [ ]:
plt.scatter(y_test, residuals)

In [ ]:
import xgboost as xg

xgbr = xg.XGBRegressor(objective = 'reg:squarederror', n_estimators = 10, seed = 123)
xgbr.fit(X_train_stand, y_train)
pred = xgbr.predict(X_test_stand)
xgb_r2 = r2_score(y_test, pred)
print("R2:", xgb_r2)